In [1]:
## import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
path = r'C:\Users\matth\05-2023 Instacart Basket Analysis'

In [3]:
## importing products merged
ords_prods_merge = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data','orders_products_mergedv2.csv'))

In [4]:
ords_prods_merge

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,...,prices,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,days_of_median,frequency_flag
0,0,0,0,0,2398795,1,2,3,7,15.0,...,9.0,Regularly busy,Least busy days,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
1,1,1,1,1,473747,1,3,3,12,21.0,...,9.0,Regularly busy,Least busy days,Most orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
2,2,2,2,2,2254736,1,4,4,7,29.0,...,9.0,Least busy,Least busy days,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
3,3,3,3,3,431534,1,5,4,15,28.0,...,9.0,Least busy,Least busy days,Most orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
4,4,4,4,4,3367565,1,6,2,7,19.0,...,9.0,Regularly busy,Regularly busy,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2447151,2447151,2447151,2447151,2447151,175901,129387,19,2,12,15.0,...,11.7,Regularly busy,Regularly busy,Most orders,23,Regular customer,8.635000,Low spender,14.0,Regular customer
2447152,2447152,2447152,2447152,2447152,1673171,129387,20,2,13,14.0,...,11.7,Regularly busy,Regularly busy,Most orders,23,Regular customer,8.635000,Low spender,14.0,Regular customer
2447153,2447153,2447153,2447153,2447153,2878236,129387,22,1,13,12.0,...,11.7,Regularly busy,Busiest days,Most orders,23,Regular customer,8.635000,Low spender,14.0,Regular customer
2447154,2447154,2447154,2447154,2447154,1330883,129387,23,1,15,14.0,...,11.7,Regularly busy,Busiest days,Most orders,23,Regular customer,8.635000,Low spender,14.0,Regular customer


In [5]:
df_customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data','customers.csv'))

## Renaming Columns

In [6]:
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
df_customer.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [8]:
df_customer.rename(columns = {'STATE' : 'State'}, inplace = True)

In [9]:
df_customer.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [10]:
df_customer.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [11]:
## checking if any changes required to data types

df_customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


## Data Quality & Consistency Checks

In [12]:
## checking if a mix in the dataset

for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

First Name


In [13]:
## updating the first name column to Str


df_customer['First Name'] = df_customer['First Name'].astype('str')

In [14]:
## double checking if ^ code worked

for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

In [15]:
## checking for blanks in dataset

df_customer.isnull().sum()

user_id           0
First Name        0
Surname           0
Gender            0
State             0
Age               0
date_joined       0
n_dependants      0
marital_status    0
income            0
dtype: int64

In [16]:
## checking for duplicates

df_dups = df_customer[df_customer.duplicated()]

In [17]:
df_customer.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Combining data set

In [18]:
ords_prods_merge.head()

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,...,prices,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,days_of_median,frequency_flag
0,0,0,0,0,2398795,1,2,3,7,15.0,...,9.0,Regularly busy,Least busy days,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
1,1,1,1,1,473747,1,3,3,12,21.0,...,9.0,Regularly busy,Least busy days,Most orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
2,2,2,2,2,2254736,1,4,4,7,29.0,...,9.0,Least busy,Least busy days,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
3,3,3,3,3,431534,1,5,4,15,28.0,...,9.0,Least busy,Least busy days,Most orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer
4,4,4,4,4,3367565,1,6,2,7,19.0,...,9.0,Regularly busy,Regularly busy,Average orders,10,New customer,6.372222,Low spender,20.5,Non-frequent customer


In [19]:
ords_prods_merge = ords_prods_merge.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2', 'Unnamed: 0.3'])

In [20]:
ords_prods_all = ords_prods_merge.merge(df_customer, on = 'user_id')

In [21]:
ords_prods_all.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,frequency_flag,First Name,Surname,Gender,State,Age,date_joined,n_dependants,marital_status,income
0,2398795,1,2,3,7,15.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,473747,1,3,3,12,21.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2254736,1,4,4,7,29.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,431534,1,5,4,15,28.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,3367565,1,6,2,7,19.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [25]:
ords_prods_all = ords_prods_all.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0.2'])

In [22]:
ords_prods_all.head()

,order_id,user_id,order_number,orders_day_of_week,hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,...,frequency_flag,First Name,Surname,Gender,State,Age,date_joined,n_dependants,marital_status,income
0,2398795,1,2,3,7,15.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,473747,1,3,3,12,21.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,2254736,1,4,4,7,29.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,431534,1,5,4,15,28.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,3367565,1,6,2,7,19.0,196,1,1,both,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [23]:
ords_prods_all.shape

(2447156, 32)

### Exporting the dataframe

In [24]:
ords_prods_all.to_csv(os.path.join(path, '02 Data','Prepared Data', 'orders_products_all.csv'))